Here we build carrier tables for MAVEs scored by a combination of Dan's calibrations and VEP methods

In [1]:
# Common imports and constants
import os
import json

import pandas as pd
from tqdm.notebook import tqdm

from cvfgaou import hailtools, gctools, notation
from cvfgaou.notation import GEQ_CHAR, LEQ_CHAR

BUCKET = os.environ["WORKSPACE_BUCKET"]

SPLICE_AI_FILTER_MAX = 0.2
AF_FILTER_MAX = 0.01 # We don't filter for rare variants in functional data, but we do for VEPs
CALIBRATION_VERSION = '2025-07-10'
DATAFRAME_VERSION = '17036510'
POINTS_DIR = f'{BUCKET}/calibration-points_2025-09-10'
RESULTS_DIR = f'{BUCKET}/combined_classes_2025-09-10'

In [2]:
# Genes of interest
genes = [
    'APP',
    'BAP1',
    'BARD1',
    'BRCA1',
    'BRCA2',
    'BRIP1',
    'CALM1',
    'CALM2',
    'CALM3',
    'GCK',
    'KCNH2',
    'KCNQ4',
    'MSH2',
    'OTC',
    'PALB2',
    'PRKN',
    'PTEN',
    'RAD51C',
    'RAD51D',
    'SCN5A',
    'SNCA',
    'TARDBP',
    'TP53',
    #'VWF'
]

In [3]:
import hail as hl
hl.init()
wgs_mt_path = os.getenv("WGS_EXOME_SPLIT_HAIL_PATH")
wgs_mt_path

In [4]:
# Load wgs
wgs_mt = hl.read_matrix_table(wgs_mt_path)
wgs_mt.describe()

In [5]:
!gsutil ls $WORKSPACE_BUCKET/cvfg_17036510

In [6]:
# Load CVFG dataframe
cvfg_df = pd.read_csv(
    f'{BUCKET}/cvfg_17036510/final_pillar_data_with_clinvar_gnomad_wREVEL_wAM_wspliceAI_expanded_090425.csv.gz',
    #index_col=0, ID is non-unique in the expanded table
    dtype={
        'Dataset': str,
        'Gene': str,
        'Chrom': str,
        #'hg38_start': int,
        #'hg38_end': int,
        'ref_allele': str,
        'alt_allele': str,
        #'auth_reported_score': float,
        'auth_reported_func_class': str
    }
)
cvfg_df

In [7]:
list(cvfg_df.columns)

In [8]:
# Filter the dataframe:

working_df = cvfg_df[
    # cvfg_df.Gene.isin(genes) & # Limit to our genes
    (cvfg_df.Flag != '*') & # Drop flagged variants (mapping errors etc.)
    ( # Splice variant filtering:
        (cvfg_df.splice_measure == 'Yes') | # Keep all variants from splice-detecting assays
        ( # Filter the rest on SpliceAI thresholds
            (cvfg_df.spliceAI_DS_AG <= SPLICE_AI_FILTER_MAX) &
            (cvfg_df.spliceAI_DS_AL <= SPLICE_AI_FILTER_MAX) &
            (cvfg_df.spliceAI_DS_DG <= SPLICE_AI_FILTER_MAX) &
            (cvfg_df.spliceAI_DS_DL <= SPLICE_AI_FILTER_MAX)
        )
    )
]

In [9]:
from cvfgaou.notation import GEQ_CHAR, LEQ_CHAR

In [10]:
# Evidence threshold mapping from integer points to labels
evidence_strength_series = pd.Series({
    nsign * points: f'{inequality} {sign}{points}'
    for nsign, sign, inequality in ((1, '+', GEQ_CHAR), (-1, '-', LEQ_CHAR))
    for points in (8,4,3,2,1)
}).sort_index(ascending=False)
#evidence_strength_series = pd.Series({
#    +8: "Pathogenic very strong",
#    +4: "Pathogenic strong",
#    +2: "Pathogenic moderate",
#    +1: "Pathogenic supporting",
#    -1: "Benign supporting",
#    -2: "Benign moderate",
#    -4: "Benign strong",
#    -8: "Benign very strong"
#})

Assign calibration points to each variant

In [14]:
# We want to build a table of the form
# contig, pos, ref, alt, points, func/vep, data source, calibration method

for blob in tqdm(gctools.list_blobs(
    f'{BUCKET}/calibrations/assays_{CALIBRATION_VERSION}/',
    return_uris=False
)):
    if blob.path.endswith('.json'):
        calibration = json.loads(blob.download_as_text())
        
        dataset = calibration['scoreset_name']

        dataset_df = working_df[working_df['Dataset'] == dataset]
        
        if dataset_df.empty:
            continue
        
        # Check how thresholds are provided. If lists, assume they should be aligned to 1, 2, 3, 4, 8
        pathogenic_thresholds = calibration['final_pathogenic_thresholds']
        if isinstance(pathogenic_thresholds, list):
            print(f'Warning: pathogenic thresholds provided as a list for {dataset}')
            pathogenic_thresholds = dict(zip([1,2,3,4,8], pathogenic_thresholds))
        
        benign_thresholds = calibration['final_benign_thresholds']
        if isinstance(benign_thresholds, list):
            print(f'Warning: benign thresholds provided as a list for {dataset}')
            benign_thresholds = dict(zip([-1,-2,-3,-4,-8], benign_thresholds))
        
        # Build series of thesholds
        thresholds = pd.concat(
            [
                pd.Series(pathogenic_thresholds),
                pd.Series(benign_thresholds)
            ]
        )
        thresholds.index = pd.to_numeric(thresholds.index)
        thresholds.sort_index(inplace=True)
        
        # Can't work without thresholds
        if thresholds.isna().all():
            continue
        
        # Directionality of thresholds
        increasing = calibration.get('inverted') == 'inverted'
        
        # Check that threholds are increasing(decreasing):
        if increasing:
            assert thresholds.dropna().is_monotonic_increasing, f'Thresholds for {dataset} are expected to be increasing: {thresholds}'
            comparisons = [pd.Series.le] * 5 + [pd.Series.ge] * 5
        else:
            assert thresholds.dropna().is_monotonic_decreasing, f'Thresholds for {dataset} are expected to be decreasing: {thresholds}'
            comparisons = [pd.Series.ge] * 5 + [pd.Series.le] * 5
        comparisons_series = pd.Series(comparisons, [-8,-4,-3,-2,-1,1,2,3,4,8])
        
        # Align thresholds, labels, and comparisons
        thresholds_df = pd.concat(
            {
                'Threshold': thresholds,
                'Comparison': comparisons_series
            },
            axis='columns'
        ).dropna(how='any').sort_index(key=lambda idx: idx.to_series().abs())
        
        result_dfs = []
        
        for gene, gene_df in dataset_df.groupby('Gene'):
            
            result_df = gene_df[
                ['Chrom', 'hg38_start', 'ref_allele', 'alt_allele']
            ].rename(columns={
                'Chrom': 'contig',
                'hg38_start': 'pos',
                'ref_allele': 'ref',
                'alt_allele': 'alt'
            })
            
            result_df['points'] = 0
            
            for points, threshold, compare in tqdm(thresholds_df.itertuples(index=True)):
                
                result_df.loc[
                    compare(gene_df['auth_reported_score'].astype(float), threshold),
                    'points'
                ] = points
            
            result_df['evidence'] = 'functional'
            result_df['dataset'] = dataset
            result_df['calibration'] = CALIBRATION_VERSION
            result_df['gene'] = gene
            
            result_dfs.append(result_df)
        
        if result_dfs:
            pd.concat(result_dfs, ignore_index=True).to_parquet(f'{POINTS_DIR}/{dataset}_{CALIBRATION_VERSION}.parquet')


In [31]:
# VEP calibrations: these will run on a simplified list of variants for each gene

vep_scores_df = working_df[
    (working_df.gnomad_MAF <= AF_FILTER_MAX) & # VEPs get filtered by AF
    # VEPS always get filtered by SpliceAI
    (working_df.spliceAI_DS_AG <= SPLICE_AI_FILTER_MAX) & 
    (working_df.spliceAI_DS_AL <= SPLICE_AI_FILTER_MAX) &
    (working_df.spliceAI_DS_DG <= SPLICE_AI_FILTER_MAX) &
    (working_df.spliceAI_DS_DL <= SPLICE_AI_FILTER_MAX) &
    # We can't score variants without AA position
    ~working_df.aa_pos.isna()
][[
    'Gene',
    'Chrom', 'hg38_start', 'ref_allele', 'alt_allele',
    'aa_ref', 'aa_pos', 'aa_alt', 'REVEL', 'AM_score'
]]

vep_scores_df['hg38_start'] = pd.to_numeric(vep_scores_df['hg38_start'], downcast='integer')
vep_scores_df['aa_pos'] = pd.to_numeric(vep_scores_df['aa_pos'], downcast='integer')

In [18]:
# Load mutpred table and filter it down to the genes of interest
mutpred_df = pd.concat(
    [
        df[df['gene_symbol'].isin(genes)]
        for df in tqdm(pd.read_table(f'{BUCKET}/mutpred2/IGVFFI7749UFOK.tsv.gz', chunksize=100000))
    ]
)

In [32]:
vep_scores_df

In [20]:
mutpred_df

In [33]:
vep_scores_df = vep_scores_df.assign(
    aa_sub=lambda df: df.apply(lambda s:f'{s.aa_ref}{s.aa_pos:d}{s.aa_alt}', axis='columns')
).merge(
    mutpred_df['gene_symbol', 'Substitution', 'MutPred2 score'],
    left_on=['Gene', 'aa_sub'],
    right_on=['gene_symbol', 'Substitution'],
    how='left'
)

In [34]:
vep_scores_df

In [9]:
# Collect calibration methods
#############################

## Load gene-specific Calibration tables

# This table is indexed by gene
gene_thresholds_dfs = {
    method_long: pd.read_csv(
        f'{BUCKET}/calibrations/pillarg_{method_short}_gene_specific_thresh.csv',
        index_col=0
    )
    for method_long, method_short in (
        ('AlphaMissense', 'AM'),
        ('MutPred2', 'MP2'),
        ('REVEL', 'REVEL')
    )
}

# Mapping from our thresholds to thresholds used in the table and the respective comparison direction
gs_threshold_map = {
    (+points if sign == '+' else -points): (f'{criterion}_{strength.title()}', comparator)
    for _, sign, criterion, _, comparator, _ in notation.DOE_TABLE
    for points, strength in notation.SOE_TABLE
}

## Global calibrations

# Thresholds from Bergquist et al. 10.1016/j.gim.2025.101402
global_thresholds = {
    'REVEL': {
        +4: lambda s: s >= 0.932, # Pathogenic strong
        +3: lambda s: s >= 0.879, # -- modetate+
        +2: lambda s: s >= 0.773, # -- moderate
        +1: lambda s: s >= 0.644, # Pathogenic supporting
        -1: lambda s: s <= 0.290, # Benign supporting
        -2: lambda s: s <= 0.183, # -- moderate
        -3: lambda s: s <= 0.052, # -- moderate+
        -4: lambda s: s <= 0.016  # -- strong
    },
    'MutPred2': {
        -4: lambda s: s <= 0.010, # Benign Strong
        -3: lambda s: s <= 0.031, # Benign Moderate+
        -2: lambda s: s <= 0.197, # Benign Moderate
        -1: lambda s: s <= 0.391, # Benign Supporting
        +1: lambda s: s >= 0.737, # Pathogenic Supporting
        +2: lambda s: s >= 0.829, # Pathogenic Moderate
        +3: lambda s: s >= 0.895, # Pathogenic Moderate+
        +4: lambda s: s >= 0.932  # Pathogenic Strong
    },
    'AlphaMissense': {
        -3: lambda s: s <= 0.070, # Benign Moderate+: <= 0.070
        -2: lambda s: s <= 0.099, # Benign Moderate: <= 0.099
        -1: lambda s: s <= 0.169, # Benign Supporting: <= 0.169
        +1: lambda s: s >= 0.792, # Pathogenic Supporting: >= 0.792
        +2: lambda s: s >= 0.906, # Pathogenic Moderate: >= 0.906
        +3: lambda s: s >= 0.972, # Pathogenic Moderate+: >= 0.972
        +4: lambda s: s >= 0.990  # Pathogenic Strong: >= 0.990
    }
}

In [12]:
# Score variants
################

predictor_cols = {
    'AlphaMissense': 'AM_score',
    'MutPred2': 'MutPred2 score',
    'REVEL': 'REVEL'
}

result_dfs = []

# Using the global method
for predictor, threshold_map in tqdm(global_thresholds.items()):
    threshold_series = pd.Series(threshold_map).sort_index(key=lambda idx: idx.to_series().abs())

    result_df = vep_scores_df[
        ['Gene', 'Chrom', 'hg38_start', 'ref_allele', 'alt_allele']
    ].renam(columns={
        'Gene': 'gene',
        'Chrom': 'contig',
        'hg38_start': 'pos',
        'ref_allele': 'ref',
        'alt_allele': 'alt'
    })
    
    result_df['points'] = 0

    for points, compare in theshold_series.items():
        result_df.loc[
            compare(vep_scores_df[predictor_cols[predictor]])
            'points'
        ] = points 
        
    result_df['evidence'] = 'predictor'
    result_df['dataset'] = predictor
    result_df['calibration'] = 'Bergquist et al. 10.1016/j.gim.2025.101402'
    result_dfs.append(result_df)


gs_threshold_map_series = pd.Series(gs_threshold_map).sort_index(key=lambda idx: idx.to_series().abs())

# Gene-specific method
for gene, gene_df in tqdm(vep_scores_df.groupby('Gene')):
    for predictor, gene_thresholds_df in gene_thresholds_dfs:
        if gene in gene_thesholds_df.index:
            
            result_df = gene_df[
                ['Chrom', 'hg38_start', 'ref_allele', 'alt_allele']
            ].rename(columns={
                'Chrom': 'contig',
                'hg38_start': 'pos',
                'ref_allele': 'ref',
                'alt_allele': 'alt'
            })
            
            result_df['points'] = 0
            
            for points, (label, comparator) in gs_thresholds_map.items():
                threshold = gene_thresholds_df.at[gene, label]
                if not np.isnan(threshold):
                    result_df.loc[
                        comparator(gene_df[predictor_cols[predictor]], threshold),
                        'points'
                    ] = points
                    
            result_df['gene'] = gene
            result_df['evidence'] = 'predictor'
            result_df['dataset'] = predictor
            result_df['calibration'] = 'gene-specific'
            
            result_dfs.append(result_df)

pd.concat(result_dfs, ignore_index=True).to_parquet(f'{POINTS_DIR}/predictors.parquet')

In [ ]:
# Sum best scores

all_scores = pd.concat(
    (pd.read_parquet(f) for f in gctools.list_blobs(POINTS_DIR)),
    ignore_index=True
)

In [ ]:
all_scores

In [ ]:
best_scores = (
    all_scores
    .group_by(['gene', 'contig', 'pos', 'ref', 'alt', 'evidence'])['points'].sort_values(key=pd.Series.abs).first()
    .group_by(['gene', 'contig', 'pos', 'ref', 'alt'])['points'].sum()

In [ ]:
best_scores

In [ ]:
# Infer points ranges
point_groups = {
    f'{GEQ_CHAR if points > 0 else LEQ_CHAR} {points:+d}':
        ((lambda x: x >= points) if points > 0 else (lambda x: x <= points))
    for points in best_scores.points.drop_duplicates()
    if points != 0
}

In [11]:
clinvar_bins_df = pd.read_csv(f'{BUCKET}/clinvar/clinvar-bins.csv.gz')

In [ ]:
exposures_file = f'{RESULTS_DIR}/exposures/combined.parquet'
clinvar_file = f'{RESULTS_DIR}/clinvar_maps/combined.parquet'
af_file = f'{RESULTS_DIR}/af_maps/combined.parquet'

clinvar_classes_dfs = []
join_af_map = {}
gene_result_dfs = []

for gene, gene_df in tqdm(best_scores.groupby('gene')):
    for classification, compare in point_groups:
                
        variant_df = gene_df[
            ['contig', 'pos', 'ref', 'alt']
        ][
            compare(gene_df.points)
        ].dropna(how='any').assign(Chromosome = lambda df: 'chr'+df['contig']).astype({'pos': int})

        if variant_df.empty:
            continue

        exposure_df, af_map, clinvar_df = hailtools.get_exposure_package(
            variant_df,
            wgs_mt,
            clinvar_bins_df,
            contig_col='Chromosome',
            pos_col='pos',
            ref_col='ref',
            alt_col='alt',
            metadata_dict={
                'Dataset': 'Combined points',
                'Gene': gene,
                'Classifier': f'Combined points',
                'Classification': classification,
                'Data Version': DATAFRAME_VERSION
            }
        )

        clinvar_classes_dfs.append(clinvar_df)
        joint_af_map.update(af_map)
        gene_result_dfs.append(exposure_df)

if clinvar_classes_dfs:
    pd.concat(clinvar_classes_dfs, ignore_index=True).to_parquet(clinvar_file)
if joint_af_map:
    pd.Series(joint_af_map).to_frame(name='AF').to_parquet(af_file)
if gene_result_dfs:
    pd.concat(gene_result_dfs, ignore_index=True).to_parquet(exposures_file)
        